In [12]:


import pandas as pd
import numpy as np
import os
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import joblib

# -----------------------------
# 1️⃣ Load models
# -----------------------------
mlp = joblib.load(r"C:\Users\vansh\OneDrive\Documents\diabetic_screening_project\notebooks\models/pima_mlp_model.pkl")
scaler = joblib.load(r"C:\Users\vansh\OneDrive\Documents\diabetic_screening_project\notebooks\models/pima_scaler.pkl")
cnn_model =  load_model(r"C:\Users\vansh\OneDrive\Documents\diabetic_screening_project\notebooks\models/efficientnetb3_aptos.h5")   # correct path

# -----------------------------
# 2️⃣ Load test CSV
# -----------------------------
test_csv_path = r"C:\Users\vansh\OneDrive\Documents\diabetic_screening_project\data\aptos\test.csv"
test_images_folder = r"C:\Users\vansh\OneDrive\Documents\diabetic_screening_project\data\aptos\test_images"

df = pd.read_csv(test_csv_path)

# Assume CSV has columns: 'PatientID', 'Image', 'Pregnancies', 'Glucose', ... (clinical features)
clinical_cols = ['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age']

results = []

for idx, row in df.iterrows():
    # --- clinical features ---
    clinical_features = np.array([row[clinical_cols].values], dtype=float)
    clinical_df = pd.DataFrame(clinical_features, columns=clinical_cols)
    clinical_scaled = scaler.transform(clinical_df)
    mlp_pred = mlp.predict(clinical_scaled)[0]

    # --- image features ---
    img_path = os.path.join(test_images_folder, row['Image'])
    if not os.path.exists(img_path):
        print(f"Image not found: {img_path}")
        continue

    img = image.load_img(img_path, target_size=(224,224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    cnn_pred = np.argmax(cnn_model.predict(img_array), axis=1)[0]

    # --- fusion logic ---
    if mlp_pred == 1 and cnn_pred >= 2:
        final_decision = "High risk: Refer to specialist"
    else:
        final_decision = "Low risk: Routine check-up"

    results.append({
        'PatientID': row['PatientID'],
        'MLP_Pred': mlp_pred,
        'CNN_Pred': cnn_pred,
        'Fusion_Decision': final_decision
    })

# -----------------------------
# 3️⃣ Convert results to DataFrame
# -----------------------------
results_df = pd.DataFrame(results)
print(results_df.head())


KeyError: "None of [Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',\n       'BMI', 'DiabetesPedigreeFunction', 'Age'],\n      dtype='object')] are in the [index]"